# AI Weather Bot

### Introduction
I wanted to make a bot to send to myself the weather every morning on my smartphone, because I am not fond of my weather app and I wanted to try making a simple bot. 

This script allowed me to send messages to myself through various platforms, for now : 
- by SMS
- by Slack

Example on what I receive every morning on slack : 
<img src ="https://alan-ai.herokuapp.com/static/img/exempleAIweather.PNG"/>

##### IMPORTANT
**What I will show here is how to build a bot who send information, not a conversational one with whom you can discuss** 

### Summary
I hope this can be useful information on many subjects and show you various interesting things on how to use API to build a simple bot : 
- How to send Bot SMS with Twilio ?
- How to send Bot messages with Slack ?
- How to scrape the weather with OpenWeatherAPI ?
- How to design a bot to send all this information ?
- Next Steps and improvements

### References

https://www.twilio.com

https://home.openweathermap.org

http://stackoverflow.com/questions/6386308/http-requests-and-json-parsing-in-python

***
## 1. How to send bot SMS with Twilio ?
Twilio is a service which allows amongst others to send SMS with a nice API. 
- Just subscribe on the developer site at https://www.twilio.com
- get the account id, the token and the number (# Find these values at https://twilio.com/user/account)
- install the python package to communicate with the API (pip install TwilioRestClient)

I put every parameters, token and ID in a json file, so that I won't disclose my personal information :)  

In [14]:
import json
with open('parameters.json') as file:
    parameters = json.load(file)

Sending a simple SMS is pretty straightforward

In [ ]:
from twilio.rest import TwilioRestClient
account_sid = parameters['twilio']['id']
auth_token = parameters['twilio']['token']
client = TwilioRestClient(account_sid, auth_token)
message = client.messages.create(to="+336XXXXXXX", from_= parameters['twilio']['from_num'],body="Hello there!")

***
## 2. How to send bot messags with Slack ?
Slack allows easy set up of bots in only two lines of codes. 

You also have to subscribe to the developer API at https://api.slack.com/bot-users

For every slack you want to set up a bot, you can subscribe and get a token you will use (that I put in my parameters json)

You also need to install the Python Slack Client (pip install SlackClient)

#### Parameters : 
You can really personalize your message : 
- The Slack channel you want to send your message to
- Of course the text
- The username
- Even the icon of the bot, here I chose a monkey

In [16]:
from slackclient import SlackClient

sc = SlackClient(parameters['slack']['token'])
sc.api_call("chat.postMessage", channel="#general", text="Hello there !",username="AI", icon_emoji=':monkey:') #icon_emoji=':robot_face:'

{u'channel': u'C23ME8Y0G',
 u'message': {u'bot_id': u'B23NKUR25',
  u'icons': {u'emoji': u':monkey:',
   u'image_64': u'https://a.slack-edge.com/d4bf/img/emoji_2015_2/apple/1f412.png'},
  u'subtype': u'bot_message',
  u'text': u'Hello there !',
  u'ts': u'1475839248.000002',
  u'type': u'message',
  u'username': u'theo'},
 u'ok': True,
 u'ts': u'1475839248.000002'}

***
## 3. How to scrape the weather with the OpenWeatherAPI ?
In this section, I will try to show how to get weather data from anywhere at any time. 

The OpenWeather API allow us to obtain this information. 

Here again, you have to subscribe to the API as developer and get a key https://openweathermap.org/ (that I put in my parameters json)

#### Scraping
Here you don't have a Python client to discuss with the API, thus we have to request directly to the API thanks to the request library. 

I personally need the weather in Paris, for which I got the City ID on the OpenWeather website, keep in mind you may have to change this ID, or you can also request according to your geocoding coordinates. 

In [18]:
openweather_key = parameters['openweather']['id']
Paris_ID = '2988507'

from urllib2 import Request, urlopen, URLError
import json

I built here a function, which request the API giving the key and the city ID.

You can change the unit type if you are not in the metric system. And again, remember you can also request according to your position. 

It gives you a big JSON full of information, I chose to keep tuples for different moment of the day (n tuples) which contain : 
- the time step (date and time)
- a quick description of the weather
- the temperature

In [21]:
def weather_tracker(api_key = openweather_key,city_ID = Paris_ID,n = 3):
    request = Request('http://api.openweathermap.org/data/2.5/forecast/city?id={0}&APPID={1}&units=metric'.format(city_ID,openweather_key))
    try:
        response = json.loads(urlopen(request).read())
        info = {}
        info['city'] = str(response['city']['name'])
        info['list'] = [(x['dt_txt'],x['weather'][0]['description'],x['main']['temp']) for x in response['list'][:n]]        
        return info
    except URLError, e:
        print("Error :",e)
        
weather = weather_tracker(n = 8)
weather

{'city': 'Paris',
 'list': [(u'2016-10-07 12:00:00', u'clear sky', 19.85),
  (u'2016-10-07 15:00:00', u'broken clouds', 19.35),
  (u'2016-10-07 18:00:00', u'overcast clouds', 16.55),
  (u'2016-10-07 21:00:00', u'light rain', 14.26),
  (u'2016-10-08 00:00:00', u'broken clouds', 12.37),
  (u'2016-10-08 03:00:00', u'few clouds', 8.37),
  (u'2016-10-08 06:00:00', u'light rain', 8.08),
  (u'2016-10-08 09:00:00', u'few clouds', 12.42)]}

For more convenient uses, I built a class to describe the weather here that I will use in the bot

It extract the interesting information for each tuple snapshot of the weather that I have in the above list. And process to some mapping : 
- extract the season from the month
- extract the period of the day (because I wanted to send different reports whether it's morning (I want today's weather) or the evening (I want tomorrow's)
- I set a scale from 0 (real bad weather with snow) to 5 (sunny) for the mood, that I extracted from the words used in the description
- For each level of the scale I associated an adjective to describe and a pretty Slack emoji to illustrate the mood

##### IMPROVEMENTS
I coded this class very fast for my personal use, it could be **enormously refined**
- I did not study every possible words in the description as I did this in the summer, there are surely a lot more
- A sentiment analysis could give great results here
- And also you could extract more information from the OpenWeatherAPI

In [22]:
class Weather_snapshot():
    def __init__(self,time,description,temperature):
        '''BASIC CONFIGURATION'''
        self.time = time
        self.description = description
        self.temperature = int(temperature)
        
        '''TIME DATA'''
        self.day = int(self.time[8:10])
        self.month = int(self.time[5:7])
        self.year = int(self.time[0:4])
        self.hour = int(self.time[11:13])
        
        #Seasons
        if self.month <= 2:
            self.season = "winter"
        elif self.month < 6:
            self.season = "spring"
        elif self.month < 10:
            self.season = "summer"
        else:
            self.season = "autumn"
            
        #Period of the day
        if (self.hour >= 4 and self.hour <= 7):
            self.period = "dawn"
        elif (self.hour > 7 and self.hour <= 11):
            self.period = "morning"
        elif (self.hour > 11 and self.hour <= 14):
            self.period = "noon"
        elif (self.hour > 14 and self.hour <= 18):
            self.period = "afternoon"
        elif (self.hour > 18 and self.hour <= 21):
            self.period = "evening"
        else:
            self.period = "night"
            
        '''WEATHER MOOD LEVELS'''
        
        if self.intersection({'snow'}):
            self.level = 1
        elif self.intersection({'rain'}):
            self.level = 2
        elif self.intersection({'cloudy','clouds','cloud'}):
            self.level = 3
        elif self.intersection({'clear'}):
            self.level = 4
        elif self.intersection({'sun','sunny'}):
            self.level = 5
        else:
            self.level = 0
        
        if self.level == 1:
            self.mood = "snowy"
            self.slack_emoji = ":snow_cloud:"
        elif self.level == 2:
            self.mood = "rainy"
            self.slack_emoji = ":rain_cloud:"
        elif self.level == 3:
            self.mood = "cloudy"
            self.slack_emoji = ":cloud:"
        elif self.level == 4:
            self.mood = "fine"
            self.slack_emoji = ":mostly_sunny:" if self.temperature < 28 else ":sunny:"
        elif self.level == 5:
            self.mood = "sunny"
            self.slack_emoji = ":sunny:" if self.temperature < 28 else ":sunny: :sunny:"
        else:
            self.mood = "uncertain"
            self.slack_emoji = ":cyclone:"
            
    def intersection(self,feature_set):
        if len(set(self.description.split(' ')).intersection(feature_set))>0:
            return True
        else:
            return False
        

***
## How to design a bot to send all this information ?

This class contains the previous functions all embedded in one object. 

The goal here is that it could allow to add more complicated information or ways to communicate (Messenger for example) in the purpose of a general Artificial Intelligence. 

##### Note : 
I copied this whole cell in a usual Python script I run automatically to send to me those Weather Reports

#### On the weather report
I use the previous class on the API extract I got before to send :
- If it's the morning : the mood and temperature right now
- If it's the evening : the mood and temperature right now + how it's going to be the next day (will the temperature be better than today ?)
Again this part is not finished and was done quick for my needs, it can be completely modified and improved


In [23]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from twilio.rest import TwilioRestClient
from slackclient import SlackClient
from urllib2 import Request, urlopen, URLError
import json

class AI():
    '''---------------------------------------------------------------------------------------------------------------'''
    '''INITIALIZATION'''
    def __init__(self,username = "Unknown",num = "+336XXXXXXX",hometown_id = '2988507'):
        '''CONFIGURATION'''
        self.config = {}
        
        #Twilio configuration
        self.config['twilio'] = {}
        self.config['twilio']['id'] = parameters['twilio']['id']
        self.config['twilio']['token'] = parameters['twilio']['token']
        self.config['twilio']['from_num'] = parameters['twilio']['from_num']
        
        #Open Weather configuration
        self.config['openweather'] = {}
        self.config['openweather']['id'] = parameters['openweather']['id']
        
        #Slack configuration
        self.config['slack'] = {}
        self.config['slack']['token'] = parameters['slack']['token']
        
        '''USER PROFILE'''
        self.user_profile = {}
        self.user_profile['username'] = username
        self.user_profile['num'] = num
        self.user_profile['hometown_id'] = hometown_id
        
        '''AI PROFILE'''
        self.name = "AI"
        
        
    '''---------------------------------------------------------------------------------------------------------------'''
    '''COMMUNICATION METHODS'''
    def send_SMS(self,body = "Hello there !",to_num = ""):
        if to_num == "":
            to_num = self.user_profile['num']
        client = TwilioRestClient(self.config['twilio']['id'], self.config['twilio']['token'])
        message = client.messages.create(to = to_num,from_ = self.config['twilio']['from_num'],body = body)
        
    def send_slack(self,body = "Hello there",channel = "#general"):
        sc = SlackClient(self.config['slack']['token'])
        sc.api_call("chat.postMessage", channel=channel, text=body,username=self.name, icon_emoji=':monkey:') #icon_emoji=':robot_face:'

    
    '''---------------------------------------------------------------------------------------------------------------'''
    '''INFORMATION METHODS'''
    def weather_tracker(self,n = 3):
        request = Request('http://api.openweathermap.org/data/2.5/forecast/city?id={0}&APPID={1}&units=metric'.format(
                self.user_profile['hometown_id'],
                self.config['openweather']['id']))
        try:
            response = json.loads(urlopen(request).read())
            info = {}
            info['city'] = str(response['city']['name'])
            info['list'] = [{'time':x['dt_txt'],
                             'description':x['weather'][0]['description'],
                             'temperature':x['main']['temp']}
                             for x in response['list'][:n]]
            info['list'] = [Weather_snapshot(**x) for x in info['list']]
            #print(info)
            return info
        except URLError, e:
            print("Error :",e)
    
    def weather_report(self,slack = False):
        weather = self.weather_tracker(n = 10)
        weather_list = weather['list']
        now = weather_list[0]
        after = weather_list[1:]

        '''SAYING HELLO'''
        if now.period in ['dawn','morning']:
            hello = "Good morning"
        elif now.period in ['noon']:
            hello = "Bon appétit"
        elif now.period in ['afternoon']:
            hello = "Good afternoon"
        elif now.period in ['evening']:
            hello = "Good evening"
        else:
            hello = "Good night"
        hello += " {0} !".format(self.user_profile['username'])
        report = ["------------------------",hello]
        
        '''CITY'''
        date = "/".join(map(str,[now.day,now.month,now.year]))
        report += ["Report in {0} on the {1}.".format(weather['city'],date)]
        
        '''NOW'''
        report += ['It is {0}:00'.format(now.hour)]
        if slack:
            report += ["The weather is {0} {1}".format(now.mood,now.slack_emoji)]
        else:
            report += ["The weather is {0}".format(now.mood)]
        report += ['The temperature is {0}°'.format(now.temperature)]
        if now.hour >= 14 or now.hour <= 4:
            tomorrow = min(after,key = lambda x:(abs(x.hour-12)))
            if now.level != tomorrow.level:
                change = "better" if now.level < tomorrow.level else "less good"
                report += ["Tomorrow, the weather will be {0}".format(change)]
                report += ["It will be {0} and the temperature {1}°".format(tomorrow.mood,tomorrow.temperature)]
            else:
                if slack:
                    report += ["Tomorrow the weather will be {0} and the temperature {1}° {2}".format(tomorrow.mood,tomorrow.temperature,tomorrow.slack_emoji)]
                else:
                    report += ["Tomorrow the weather will be {0} and the temperature {1}°".format(tomorrow.mood,tomorrow.temperature)]
            
        return "\n".join(report)
    
    def send_weather(self,methods = ["SMS"]):
        print("Sending weather report ...")
        for method in methods:
            if method == "SMS":
                self.send_SMS(self.weather_report())
            elif method == "slack":
                self.send_slack(self.weather_report(slack = True),channel = "#weather")
        print('Sending weather report OK"')

##### HOW TO USE IT ?
First you instantiate the AI for you giving it the important information

In [24]:
AI = AI(username = 'Theo')

You can simply send the weather like that, giving just the way you prefer. 

In my slack I created a "#weather" channel where every message go to (but you can change it in the class)

In [25]:
AI.send_weather(["slack"])

Sending weather report ...
Sending weather report OK"


You can also just send regular messages with the other methods

In [26]:
AI.send_slack("TEST",channel = "#random")

***
## NEXT STEPS AND IMPROVEMENTS
I already talked about what could be improved
- in the weather report
- in how much information you extract
- by adding other channels like Messenger to the bot
- but also and the most important one, by using that to build a more general AI Bot with whom you can talk
- for now I am lauching automatically this script with a Windows batch file (.bat), but it could be interesting to set it up with AWS Lambda or even PythonAnywhere